# Sentinel-1 데이터 검색 예제

이 노트북은 ASF Data Search를 사용하여 Sentinel-1 SLC 데이터를 검색하는 예제입니다.

## 목표
- 포항/경주 지역의 Sentinel-1 데이터 검색
- 검색 결과 시각화
- 다운로드 가능한 제품 목록 확인

In [ ]:
# 필요한 라이브러리 임포트
import sys
sys.path.append('../')

from src.data_retrieval import Sentinel1Retriever
from src.config import get_config
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

print("✓ 라이브러리 로드 완료")

## 1단계: 설정 확인

프로젝트 설정 파일을 로드하고 검색 영역(AOI)을 확인합니다.

In [ ]:
# 설정 로드
config = get_config()

# AOI 정보 확인
aoi = config.get('aoi')
print("검색 영역 (AOI):")
print(f"  이름: {aoi['name']}")
print(f"  경도 범위: {aoi['min_lon']} ~ {aoi['max_lon']}")
print(f"  위도 범위: {aoi['min_lat']} ~ {aoi['max_lat']}")

# Sentinel-1 검색 파라미터
s1_config = config.get('sentinel1')
print("\nSentinel-1 검색 조건:")
print(f"  플랫폼: {s1_config['platform']}")
print(f"  제품 타입: {s1_config['product_type']}")
print(f"  센서 모드: {s1_config['sensor_mode']}")
print(f"  기간: {s1_config['date_range']['start']} ~ {s1_config['date_range']['end']}")

## 2단계: 데이터 검색

ASF Data Search를 사용하여 Sentinel-1 데이터를 검색합니다.

In [ ]:
# Retriever 초기화
retriever = Sentinel1Retriever()

# 데이터 검색 (2023년 1월 한 달간)
print("데이터 검색 중...")
products_df = retriever.search_products(
    start_date='2023-01-01',
    end_date='2023-01-31',
    max_results=50
)

print(f"\n✓ 검색 완료: {len(products_df)}개 제품 발견")

# 검색 결과 테이블 표시
if not products_df.empty:
    retriever.display_products(products_df)

## 3단계: 검색 결과 분석

검색된 데이터의 시간 분포와 통계를 확인합니다.

In [ ]:
if not products_df.empty:
    # 시간별 분포
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 날짜별 제품 수
    products_df['acquisition_date'].value_counts().sort_index().plot(
        kind='bar',
        ax=axes[0],
        color='steelblue'
    )
    axes[0].set_title('날짜별 제품 수')
    axes[0].set_xlabel('날짜')
    axes[0].set_ylabel('제품 수')
    axes[0].tick_params(axis='x', rotation=45)
    
    # 궤도 방향별 분포
    if 'orbit_direction' in products_df.columns:
        products_df['orbit_direction'].value_counts().plot(
            kind='pie',
            ax=axes[1],
            autopct='%1.1f%%',
            colors=['skyblue', 'lightcoral']
        )
        axes[1].set_title('궤도 방향별 분포')
        axes[1].set_ylabel('')
    
    plt.tight_layout()
    plt.show()
    
    # 통계 정보
    print("\n=== 검색 결과 통계 ===")
    print(f"총 제품 수: {len(products_df)}개")
    print(f"검색 기간: {products_df['acquisition_date'].min()} ~ {products_df['acquisition_date'].max()}")
    if 'size_mb' in products_df.columns:
        total_size_gb = products_df['size_mb'].sum() / 1024
        print(f"총 데이터 크기: {total_size_gb:.2f} GB")
else:
    print("검색 결과가 없습니다.")

In [ ]:
if not products_df.empty:
    # 다운로드할 제품 선택 (예: 첫 2개)
    selected_products = products_df.head(2)
    
    print("다운로드 대상 제품:")
    print("=" * 80)
    for idx, row in selected_products.iterrows():
        print(f"\n제품 {idx + 1}:")
        print(f"  이름: {row['title']}")
        print(f"  날짜: {row['acquisition_date']}")
        print(f"  크기: {row.get('size_mb', 0):.2f} MB")
        print(f"  다운로드 URL: {row['url']}")
    
    print("\n" + "=" * 80)
    print("\n💡 다운로드 방법:")
    print("1. 터미널에서 실행:")
    print("   python run_data_search.py --start-date 2023-01-01 --end-date 2023-01-31 --download --max-products 2")
    print("\n2. 또는 아래 코드로 직접 다운로드:")
    print("   # downloaded = retriever.download_products(selected_products)")
else:
    print("다운로드할 제품이 없습니다.")

## 다음 단계

1. **인증 설정**: `.netrc` 파일 설정 (상위 디렉토리에서 `python scripts/setup_netrc.py` 실행)
2. **데이터 다운로드**: 위 명령어로 실제 다운로드 수행
3. **전처리**: 다운로드한 SLC 데이터 전처리
4. **간섭도 생성**: InSAR 처리 수행

자세한 내용은 프로젝트 `README.md`를 참고하세요!